In [1]:
! pip install ipykernel --upgrade
! git clone https://github.com/weizequan/TASHR

Cloning into 'TASHR'...
remote: Enumerating objects: 379, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 379 (delta 63), reused 154 (delta 55), pack-reused 215 (from 1)
Receiving objects: 100% (379/379), 18.05 MiB | 3.79 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [4]:
#! pip install tensorflow-cpu
! chmod +x ./TASHR/ocr/setup.sh && ./TASHR/ocr/setup.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 6.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 4.9 MB/s eta 0:00:0000:0100:01
  Using cached pyparsing-3.1.4-py3-none-any.whl (104 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 MB 5.4 MB/s eta 0:00:0000:0100:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.6/189.6 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [5]:
! python TASHR/lite_train.py

The TensorFlow library was compiled to use AVX instructions, but these aren't available on your machine.
^C
